# SCaling and Queues


## reference

https://kubernetes.io/docs/tasks/job/fine-parallel-processing-work-queue/


In this example, we will run a Kubernetes Job with multiple parallel worker processes in a given pod.

In this example, as each pod is created, it picks up one unit of work from a task queue, processes it, and repeats until the end of the queue is reached.


1. Start a storage service to hold the work queue.
2. Create a queue, and fill it with messages. 
3. Start a Job that works on tasks from the queue. 


In [39]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [40]:
! $k get all

NAME                        READY   STATUS    RESTARTS   AGE
pod/job-wq-2-dnpsn          1/1     Running   0          53s
pod/job-wq-2-lfg2h          1/1     Running   0          53s
pod/redis-master            1/1     Running   0          7h2m
pod/temp-5878477f6c-zxqvx   1/1     Running   0          7h5m

NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
service/kubernetes   ClusterIP   10.152.183.1     <none>        443/TCP    7h35m
service/redis        ClusterIP   10.152.183.144   <none>        6379/TCP   7h1m

NAME                   READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/temp   1/1     1            1           7h5m

NAME                              DESIRED   CURRENT   READY   AGE
replicaset.apps/temp-5878477f6c   1         1         1       7h5m

NAME                 COMPLETIONS   DURATION   AGE
job.batch/job-wq-2   0/1 of 2      53s        53s


In [41]:
! microk8s.status |grep dns

dns: enabled


In [42]:
! mkdir -p redis

In [43]:
! (cd redis; wget https://kubernetes.io/examples/application/job/redis/redis-pod.yaml)

--2020-03-30 21:32:59--  https://kubernetes.io/examples/application/job/redis/redis-pod.yaml
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240 [application/x-yaml]
Saving to: ‘redis-pod.yaml.1’

redis-pod.yaml.1    100%[===================>]     240  --.-KB/s    in 0s      

2020-03-30 21:33:00 (7.06 MB/s) - ‘redis-pod.yaml.1’ saved [240/240]



In [44]:
! ls -l redis

total 36
-rw-rw-r-- 1 ubuntu ubuntu  115 Mar 30 14:23 Dockerfile
-rw-rw-r-- 1 ubuntu ubuntu  195 Mar 30 15:43 Makefile
-rw-rw-r-- 1 ubuntu ubuntu  235 Mar 30 15:48 job.yaml
-rw-rw-r-- 1 ubuntu ubuntu  240 Mar 30 14:22 redis-pod.yaml
-rw-rw-r-- 1 ubuntu ubuntu  240 Mar 30 21:33 redis-pod.yaml.1
-rw-rw-r-- 1 ubuntu ubuntu  135 Mar 30 14:23 redis-service.yaml
-rw-rw-r-- 1 ubuntu ubuntu 5545 Mar 30 14:24 rediswq.py
-rw-rw-r-- 1 ubuntu ubuntu  657 Mar 30 14:24 worker.py


In [45]:
%%bash

cd redis
wget https://kubernetes.io/examples/application/job/redis/redis-service.yaml
wget https://kubernetes.io/examples/application/job/redis/Dockerfile
wget https://kubernetes.io/examples/application/job/redis/job.yaml
wget https://kubernetes.io/examples/application/job/redis/rediswq.py
wget https://kubernetes.io/examples/application/job/redis/worker.py

--2020-03-30 21:33:00--  https://kubernetes.io/examples/application/job/redis/redis-service.yaml
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135 [application/x-yaml]
Saving to: ‘redis-service.yaml.1’

     0K                                                       100% 7.45M=0s

2020-03-30 21:33:00 (7.45 MB/s) - ‘redis-service.yaml.1’ saved [135/135]

--2020-03-30 21:33:00--  https://kubernetes.io/examples/application/job/redis/Dockerfile
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115 [text/plain]
Saving to: ‘Dockerfile.1’

     0K                                                       100% 7.01M=0s

2020-03-30 21:33:00 (7.01 MB/s) - ‘Dockerfile.1’ saved [115/115]

--2020-03-30 21:33:00--  https://kuber

In [46]:
! ls -l redis

total 60
-rw-rw-r-- 1 ubuntu ubuntu  115 Mar 30 14:23 Dockerfile
-rw-rw-r-- 1 ubuntu ubuntu  115 Mar 30 21:33 Dockerfile.1
-rw-rw-r-- 1 ubuntu ubuntu  195 Mar 30 15:43 Makefile
-rw-rw-r-- 1 ubuntu ubuntu  235 Mar 30 15:48 job.yaml
-rw-rw-r-- 1 ubuntu ubuntu  244 Mar 30 21:33 job.yaml.1
-rw-rw-r-- 1 ubuntu ubuntu  240 Mar 30 14:22 redis-pod.yaml
-rw-rw-r-- 1 ubuntu ubuntu  240 Mar 30 21:33 redis-pod.yaml.1
-rw-rw-r-- 1 ubuntu ubuntu  135 Mar 30 14:23 redis-service.yaml
-rw-rw-r-- 1 ubuntu ubuntu  135 Mar 30 21:33 redis-service.yaml.1
-rw-rw-r-- 1 ubuntu ubuntu 5545 Mar 30 14:24 rediswq.py
-rw-rw-r-- 1 ubuntu ubuntu 5545 Mar 30 21:33 rediswq.py.1
-rw-rw-r-- 1 ubuntu ubuntu  657 Mar 30 14:24 worker.py
-rw-rw-r-- 1 ubuntu ubuntu  657 Mar 30 21:33 worker.py.1


# Run the redis cli interactively

in a terminal windo do this

## kubectl run -i --tty temp --image redis --command "/bin/sh"

# OOPS - need a basic redis service up

In [47]:
! $k create -f ./redis/redis-pod.yaml

Error from server (AlreadyExists): error when creating "./redis/redis-pod.yaml": pods "redis-master" already exists


In [48]:
! $k create -f ./redis/redis-service.yaml

Error from server (AlreadyExists): error when creating "./redis/redis-service.yaml": services "redis" already exists


In [90]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [91]:
! $k delete -f ./redis/job.yaml

job.batch "job-wq-2" deleted


In [92]:
#! $k --help

In [93]:
! $k create -f ./redis/job.yaml

job.batch/job-wq-2 created


In [94]:
! $k get pods

NAME                    READY   STATUS              RESTARTS   AGE
job-wq-2-d4kgv          1/1     Terminating         0          5m6s
job-wq-2-dpb8l          0/1     ContainerCreating   0          0s
job-wq-2-rzqn5          0/1     ContainerCreating   0          0s
job-wq-2-skml8          1/1     Terminating         0          5m6s
redis-master            1/1     Running             0          7h9m
temp-5878477f6c-zxqvx   1/1     Running             0          7h12m


In [95]:
! $k describe jobs/job-wq-2

Name:           job-wq-2
Namespace:      default
Selector:       controller-uid=8b31deed-cc5e-45f9-9906-5f0710f0ceb1
Labels:         controller-uid=8b31deed-cc5e-45f9-9906-5f0710f0ceb1
                job-name=job-wq-2
Annotations:    <none>
Parallelism:    2
Completions:    <unset>
Start Time:     Mon, 30 Mar 2020 21:40:28 +0000
Pods Statuses:  2 Running / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  controller-uid=8b31deed-cc5e-45f9-9906-5f0710f0ceb1
           job-name=job-wq-2
  Containers:
   c:
    Image:        tbutzer/job-wq-2
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From            Message
  ----    ------            ----  ----            -------
  Normal  SuccessfulCreate  0s    job-controller  Created pod: job-wq-2-rzqn5
  Normal  SuccessfulCreate  0s    job-controller  Created pod: job-wq-2-dpb8l


In [96]:
#!$k logs pods/job-wq-2-25zcv

In [97]:
#!$k logs pods/job-wq-2-nj7wb

In [104]:
! sleep 15

In [105]:
%%bash

POD=$(microk8s.kubectl get pods  -o name |grep wq-2)
echo $POD
#$k get logs pod ${POD}

for pod in $POD; do echo . ; $k logs $pod; echo ------;  done

pod/job-wq-2-dpb8l pod/job-wq-2-rzqn5
.
------
.
------


In [106]:
! $k get pods

NAME                    READY   STATUS    RESTARTS   AGE
job-wq-2-dpb8l          1/1     Running   0          77s
job-wq-2-rzqn5          1/1     Running   0          77s
redis-master            1/1     Running   0          7h11m
temp-5878477f6c-zxqvx   1/1     Running   0          7h13m


In [115]:
! $k logs pods/job-wq-2-rzqn5 
